In [10]:
import pandas as pd
import os
import requests
import json
from config import password
from sqlalchemy import create_engine

In [6]:
def import_func(country_code):
    
    #Creating Path
    
    path = os.path.join('data',f'{country_code}_youtube_trending_data.csv')
    
    
    #Storing dataframe to df
    
    df=pd.read_csv(path)
    
    #Remving unwanted columns below
    
    df=df[['video_id','title','publishedAt','channelTitle','categoryId','trending_date','view_count','likes','dislikes','comment_count','thumbnail_link']]
    
    
    #Changing object types to date types for two columns
    
    df['publishedAt']=pd.to_datetime(df['publishedAt'])
    df['trending_date']=pd.to_datetime(df['trending_date'])
    
    
    #Removing time stamp from date
    
    df['publishedAt']=df['publishedAt'].dt.date 
    df['trending_date']=df['trending_date'].dt.date
    
    
    with open(f'data/{country_code}_category_id.json', 'r') as read_file:
        category_ids = json.load(read_file)

        df=df.astype({'categoryId': 'str'})
        
    for index,row in df.iterrows():
    
        for entry in category_ids["items"]:

            if row["categoryId"]==entry["id"]:
                df.at[index,"categoryId"]=entry["snippet"]["title"]
    
    #Adding country Code as column
    df['country']=f'{country_code}'
    col_name='country'
    
    
    #Moving country code to first column
    first_col = df.pop(col_name)
    df.insert(0,col_name,first_col)
    
    return df

In [7]:
#USA Dataframe
df_us=import_func('US')

#Brasil Dataframe
df_br=import_func('BR')

#Canada Dataframe
df_ca=import_func('CA')

#Mexico Dataframe
df_mx=import_func('MX')

In [8]:
#Creating a varible to add all dfs
country_df=[df_us,df_br,df_ca,df_mx]

df_main = pd.concat(country_df)

In [11]:
# #Connect to Database

pg_user = 'postgres'
password=password
db_name = 'youtube_database'
connection_string = f'{pg_user}:{password}@localhost:5432/{db_name}'
engine=create_engine(f'postgresql://{connection_string}')

#Connect to Database - Alternative
#dbuser = 'postgres'
#dbpassword = 'Sm6Jc5bqbiNQdsVAo7eN'
#dbhost = 'localhost'
#dbport = '5432'
#dbname= 'youtube_database'
#connection_string2 = f'{dbuser}:{dbpassword}@database-1.cvmfiiilpm7y.us-east-1.rds.amazonaws.com:{dbport}/{dbname}'
#engine=create_engine(f'postgresql://{connection_string2}')

In [12]:
engine.table_names()

['youtube_table']

In [14]:
df_main.to_sql(name='youtube_table',con=engine,if_exists='append',index=False)